Setup

In [2]:
import numpy as np
import pandas as pd
import re

df = pd.read_csv('data/cars_datasets_2025.csv', encoding='latin-1')
pd.set_option('display.float_format', '{:,.2f}'.format)

View Data

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.sample(20)

In [ ]:
df.info()

In [ ]:
df.describe()

Check for empty columns

In [ ]:
df.isnull().sum()

Drop Empty columns

In [3]:
empty_columns = df[df.isna().any(axis=1)]
df = df.drop(empty_columns.index)

Check for duplicates

In [ ]:
df.duplicated().sum()
df[df.duplicated()]

Drop Duplicates

In [4]:
df = df.drop_duplicates(keep="first")


Rename columns for easier referencing

In [ ]:
print(df.columns)

new_columns = ['car_brand', 'car_name', 'engine', 'cc_battery_capacity', 'horsepower', 'total_speed_kmh', '0_100_kmh_performance_secs', 'price', 'fuel_type', 'seats', 'torque_nm']

df.columns = new_columns

print(df.columns)

Split cars with multiple engine types into different rows

In [6]:
df = df.reset_index(names="og_id")

cc_re = re.compile(r'([\d.,]+)\s*cc\b', flags=re.I) # find cc values
kwh_range = re.compile(r'([\d.,]+)\s*[-–]\s*([\d.,]+)\s*kwh\b', re.I) # match kwh ranges: 'x-y kwh'
kwh_re = re.compile(r'([\d.,]+)\s*kwh\b', flags=re.I) # match single kwh value: 'x kwh'
num_re = re.compile(r'([\d.,]+)') # any number that hasn't already been matched
paren_re = re.compile(r'\([^)]*\)') 
iv_token = re.compile(r'\b[ivx]\s*\d\b', re.I)

general_norm = re.compile(r'\s*(?:/|,(?=\s*[A-Za-z])|-(?!\s*\d+\s*kwh))\s*', re.I)
engine_norm = re.compile(r'\s*/\s*', re.I)
fuel_norm = re.compile(r'\s*(?:/|,(?=\s*[A-Za-z]))\s*', re.I) # ensure fuel doesn't get split by dashes

cols_to_split = [
    "engine",
    "cc_capacity",
    "kwh_capacity",
    "horsepower",
    "0_100_kmh_performance_secs",
    "fuel_type",
    "torque_nm"
]

# remove commas from regex matches and turn to float
def _nums(s):
    out = []
    for m in s:
        try:
            out.append(float(m.replace(',', '')))
        except Exception:
            pass
    return out

def extract_capacity_lists(val):
    s = str(val)
    s_lower = s.lower()
    s_norm = kwh_range.sub(r'\1 kwh / \2 kwh', s_lower) # turn kwh ranges from 'x-y kwh' to 'x kwh' / 'y kwh' for easier splitting later

    # place kwh values into a list
    kwh_nums = _nums(kwh_re.findall(s_norm))
    kwh_list = kwh_nums if kwh_nums else [pd.NA]
    
    # prepare non-kwh matches
    no_kwh = kwh_re.sub("", s_norm)
    no_kwh = paren_re.sub(" ", no_kwh) # strip parenthesis its contents
    no_kwh = iv_token.sub(" ", no_kwh) # strip any i or v matches

    cc_nums = _nums(cc_re.findall(no_kwh)) # find cc values
    no_kwh_cc = cc_re.sub("", no_kwh) # non-kwh/cc matches

    cc_fallback = _nums(num_re.findall(no_kwh_cc)) # if no cc or kwh attached to value, assume it's cc
    cc_all = (cc_nums + cc_fallback) if (cc_nums or cc_fallback) else [pd.NA]

    return pd.Series([cc_all, kwh_list])

def pre_engine(s):
    if pd.isna(s):
        return s
    t = str(s)
    t = re.sub(r'\b(?:or|vs)\b', ' / ', t, flags=re.I) # standardize splitting engines with '/' instead of 'or' or 'vs'
    t = re.sub(r'(?<!\d)\s*,\s*(?!\d)', ' ', t) # ignores commas in numbers
    t = re.sub(r'\binline\s*[- ]?\s*(\d)\b', r'inline-\1', t, flags=re.I) # standardize 'inline 4' as 'inline-4'
    t = re.sub(r'\bi\s*[- ]?\s*(\d)\b', r'i\1', t, flags=re.I) # standardize 'i 4' as 'i4'
    t = re.sub(r'\bv\s*[- ]?\s*(\d)\b', r'v\1', t, flags=re.I) # standardize 'v 6' as 'v6'
    t = re.sub(r'\s*/\s*', ' / ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

# normalize splitting rows with '/'
def normalize_and_split(x, norm):
    if isinstance(x, list):
        return x
    s = re.sub(norm, '/', str(x)) # replace separators with '/'
    parts = [p.strip() for p in s.split('/') if p.strip()] # strip then split on '/'
    return parts if parts else [pd.NA]

def _clean_parts(lst):
    if not isinstance(lst, list):
        return lst
    return [re.sub(r'\s*,\s*$', '', p).strip() for p in lst]

# create cc and kwh capacity columns from cc_batter_capacity column
df[["cc_capacity", "kwh_capacity"]] = df["cc_battery_capacity"].apply(extract_capacity_lists)
df = df.drop(columns=["cc_battery_capacity"]) # drop original column

df["engine"] = df["engine"].map(pre_engine)
df["engine"] = df["engine"].map(lambda v: normalize_and_split(v, engine_norm))
df["engine"] = df["engine"].map(_clean_parts)

for col in cols_to_split:
    if col in ("cc_capacity", "kwh_capacity", "engine"):
        continue
    elif col == "fuel_type":
        pat = fuel_norm
    else:
        pat = general_norm
    df[col] = df[col].map(lambda v: normalize_and_split(v, pat))

df["_max_splits"] = df[cols_to_split].apply(lambda r: max(len(r[c]) for c in cols_to_split), axis=1) # determine how many rows to make
df = df.loc[df.index.repeat(df["_max_splits"].where(df["_max_splits"] > 1, 1))].copy()
df["_idx"] = df.groupby("og_id").cumcount()
df["_idx"] = pd.to_numeric(df["_idx"], errors="coerce").fillna(-1).astype("Int64")

def take_n_request(row, col):
    vals = row[col]
    j = min(row["_idx"], len(vals) - 1)
    return vals[j]

for col in cols_to_split:
    df[col] = df.apply(take_n_request, axis=1, col=col) #???

# create row_id column consisting of "og_id" and "_idx" to highlight highlight original row and what variant it is
df["row_id"] = df["og_id"].astype(str) + "-" + df["_idx"].astype(str)
df.insert(0, "row_id", df.pop("row_id"))

df = df.drop(columns=["_max_splits"])

Address odd characters

In [7]:
def scrub_ascii(x):
    if isinstance(x, str):
        return re.sub(r'[^\x00-\x7F]+', '-', x)
    return x

obj_cols = df.select_dtypes(include="object").columns
for col in obj_cols:
    df[col] = df[col].map(scrub_ascii)

Remove non-numerical characters in numerical columns

In [8]:
df['price'] = (df['price'].str.replace('$', '', case=False).str.replace(',', '', case=False)).str.strip()

numeric_cols = ["horsepower", "torque_nm", "0_100_kmh_performance_secs", "total_speed_kmh"]

for col in numeric_cols:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace(r'(\d)\.\s+(\d)', r'\1.\2', regex=True)
    df[col] = df[col].str.replace(r'[A-Za-z/ ]+', '', regex=True)
    df[col] = df[col].str.replace(r'[^0-9\.\-]', '', regex=True)
    df[col] = df[col].str.strip()

Address range columns

In [9]:
for col in ["price", "horsepower", "torque_nm", "0_100_kmh_performance_secs"]:
    df[col] = df[col].astype(str).str.replace(',', '', regex=False).str.strip()
    
range_re = re.compile(r'-?\d+(?:\.\d+)?')

def extract_range(val):
    if pd.isna(val):
        return pd.Series([np.nan, np.nan])
    
    s = str(val)
    nums = range_re.findall(s)
    nums = [float(n) for n in nums] 
    
    if len(nums) == 0:
        return pd.Series([np.nan, np.nan])
    elif len(nums) == 1:
        return pd.Series([nums[0], nums[0]])
    else:
        a, b = nums[0], nums[1]
        return pd.Series([min(a, b), max(a, b)])
    
def extract_range_performance(val):
    if pd.isna(val):
        return pd.Series([np.nan, np.nan])
    
    s = str(val)
    nums = range_re.findall(s)
    nums = [float(n) for n in nums]

    if len(nums) == 0:
        return pd.Series([np.nan, np.nan])
    
    realistic = [n for n in nums if n > 0]

    if len(realistic) == 0:
        realistic = nums

    if len(realistic) == 1:
        return pd.Series([realistic[0], realistic[0]])
    else:
        a, b = realistic[0], realistic[1]
        return pd.Series([min(a, b), max(a, b)])
    
for col in ["price", "horsepower", "torque_nm"]:
    new_cols = [f"{col}_min", f"{col}_max"]
    df[new_cols] = df[col].apply(extract_range)

df[["0_100_kmh_performance_secs_min",
    "0_100_kmh_performance_secs_max"]] = (
        df["0_100_kmh_performance_secs"].apply(extract_range_performance)
    )

df = df.drop(columns=["price", "horsepower", "torque_nm", "0_100_kmh_performance_secs"])

Address Seats Column

In [10]:
def format_seats(val):
    if pd.isna(val):
        return pd.Series([np.nan, np.nan])
    
    s = str(val).strip()
    if re.fullmatch(r'\d+', s):
        n = int(s)
        return pd.Series([n,n])
    
    add_seats = re.fullmatch(r'(\d+)\+(\d+)', s)
    if add_seats:
        a, b = int(add_seats.group(1)), int(add_seats.group(2))
        total = a + b
        return pd.Series([total, total])
    
    range_seats = re.fullmatch(r'(\d+)\s*-\s*(\d+)', s)
    if range_seats:
        a, b = int(range_seats.group(1)), int(range_seats.group(2))
        return pd.Series([min(a, b), max(a, b)])
    
    return pd.Series([np.nan, np.nan])

df[['seats_min', 'seats_max']] = df['seats'].apply(format_seats)
df = df.drop(columns=["seats"])

Convert columns to numerical type

In [11]:
numeric_cols = [
    "total_speed_kmh", 
    "seats_min", "seats_max",
    "cc_capacity", "kwh_capacity",
    "price_min", "price_max",
    "horsepower_min", "horsepower_max",
    "torque_nm_min", "torque_nm_max",
    "0_100_kmh_performance_secs_min", "0_100_kmh_performance_secs_max"
]

df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

Check Fuel-Type Categories

In [ ]:
df['fuel_type'].value_counts()

Normalize and Drop Fuel Type Categories

In [13]:
df['fuel_type'] = df['fuel_type'].str.lower().str.strip()

df['fuel_type'] = df['fuel_type'].replace({
    'electric': 'electric',
    'ev': 'electric',
    'plug-in': 'electric',
    'hybrid': 'hybrid',
    'plug in hybrid': 'hybrid',
    'plug-in hybrid': 'hybrid',
    'plug in hyrbrid': 'hybrid',
    'hybrid (gas + electric)': 'hybrid',
    'hybrid (petrol)': 'hybrid',
    'petrol': 'petrol',
    'gas': 'petrol',
    'awd': 'petrol',
    'diesel': 'diesel',
    'hydrogen': 'hydrogen',
    'cng': 'cng',
})

# only 3 hydrogen and 1 CNG row, not enough to warrant having its own fuel type category so I thought it was best to omit it
drop_fuel_types = df[df['fuel_type'].str.lower().isin(['cng', 'hydrogen'])]

df = df.drop(drop_fuel_types.index)

Check Car Brands

In [17]:
df['car_brand'].unique()

array(['Ferrari', 'Rolls Royce', 'Ford', 'Mercedes', 'Audi', 'Bmw',
       'Aston Martin', 'Bentley', 'Lamborghini', 'Toyota', 'Nissan',
       'Volvo', 'Kia', 'Honda', 'Hyundai', 'Mahindra', 'Maruti Suzuki',
       'Volkswagen', 'Porsche', 'Cadillac', 'Tata Motors', 'Tesla',
       'Jeep', 'Mazda', 'Chevrolet', 'Gmc', 'Peugeot', 'Bugatti',
       'Jaguar Land Rover', 'Acura', 'Mitsubishi'], dtype=object)

Fix Car Brand Capitalization and Create New Columns

In [16]:
df['car_brand'] = (
    df['car_brand']
    .str.strip()
    .str.lower()
    .str.title()
)

# assign region to each car brand
region_map = {
    'Volkswagen': 'Europe', 
    'Porsche': 'Europe', 
    'Peugeot': 'Europe', 
    'Jaguar Land Rover': 'Europe', 
    'Bmw': 'Europe', 
    'Lamborghini': 'Europe', 
    'Audi': 'Europe', 
    'Mercedes': 'Europe', 
    'Rolls Royce': 'Europe', 
    'Volvo': 'Europe',
    'Aston Martin': 'Europe', 
    'Bugatti': 'Europe', 
    'Ferrari': 'Europe', 
    'Bentley': 'Europe',
    'Nissan': 'Asia', 
    'Mazda': 'Asia', 
    'Kia': 'Asia', 
    'Toyota': 'Asia', 
    'Mitsubishi': 'Asia', 
    'Hyundai': 'Asia', 
    'Acura': 'Asia', 
    'Honda': 'Asia', 
    'Maruti Suzuki': 'Asia', 
    'Tata Motors': 'Asia', 
    'Mahindra': 'Asia',
    'Ford': 'North America', 
    'Gmc': 'North America', 
    'Chevrolet': 'North America', 
    'Tesla': 'North America', 
    'Cadillac': 'North America', 
    'Jeep': 'North America'
}

# create new column for car brand region
df['region'] = df['car_brand'].map(region_map).fillna('Other')

country_map = {
    'Ferrari': 'Italy',
    'Rolls Royce': 'United Kingdom',
    'Ford': 'United States',
    'Mercedes': 'Germany',
    'Audi': 'Germany',
    'Bmw': 'Germany',
    'Aston Martin': 'United Kingdom',
    'Bentley': 'United Kingdom',
    'Lamborghini': 'Italy',
    'Toyota': 'Japan',
    'Nissan': 'Japan',
    'Volvo': 'Sweden',
    'Kia': 'South Korea',
    'Honda': 'Japan',
    'Hyundai': 'South Korea',
    'Mahindra': 'India',
    'Maruti Suzuki': 'India',
    'Volkswagen': 'Germany',
    'Porsche': 'Germany',
    'Cadillac': 'United States',
    'Tata Motors': 'India',
    'Tesla': 'United States',
    'Jeep': 'United States',
    'Mazda': 'Japan',
    'Chevrolet': 'United States',
    'Gmc': 'United States',
    'Peugeot': 'France',
    'Bugatti': 'France',
    'Jaguar Land Rover': 'United Kingdom',
    'Acura': 'Japan',
    'Mitsubishi': 'Japan'
}

# assign country to each car brand
country_map = {
    'Ferrari': 'Italy',
    'Rolls Royce': 'United Kingdom',
    'Ford': 'United States',
    'Mercedes': 'Germany',
    'Audi': 'Germany',
    'Bmw': 'Germany',
    'Aston Martin': 'United Kingdom',
    'Bentley': 'United Kingdom',
    'Lamborghini': 'Italy',
    'Toyota': 'Japan',
    'Nissan': 'Japan',
    'Volvo': 'Sweden',
    'Kia': 'South Korea',
    'Honda': 'Japan',
    'Hyundai': 'South Korea',
    'Mahindra': 'India',
    'Maruti Suzuki': 'India',
    'Volkswagen': 'Germany',
    'Porsche': 'Germany',
    'Cadillac': 'United States',
    'Tata Motors': 'India',
    'Tesla': 'United States',
    'Jeep': 'United States',
    'Mazda': 'Japan',
    'Chevrolet': 'United States',
    'Gmc': 'United States',
    'Peugeot': 'France',
    'Bugatti': 'France',
    'Jaguar Land Rover': 'United Kingdom',
    'Acura': 'Japan',
    'Mitsubishi': 'Japan'
}

# create new column for car brand country
df['country'] = df['car_brand'].map(country_map).fillna('Other')

Create Price Tier Column

In [ ]:
bins = [0, 25000, 50000, 100000, 250000, float('inf')]
labels = ['Budget', 'Midrange', 'Premium', 'Luxury', 'Supercar']

df['price_tier'] = pd.cut(df['price_max'], bins=bins, labels=labels, right=True)

df.groupby('price_tier', observed=True).agg(
    avg_price=('price_max', 'mean'),
    avg_hp=('horsepower_max', 'mean'),
    avg_performance=('0_100_kmh_performance_secs_min', 'mean'),
    avg_torque=('torque_nm_max', 'mean'),
    avg_cc_capacity=('cc_capacity', 'mean'),
    avg_total_speed=('total_speed_kmh', 'mean'),
    avg_seats=('seats_max','mean'),
    diesel=('fuel_type', lambda x: (x.str.lower() == 'diesel').sum()),
    petrol=('fuel_type', lambda x: (x.str.lower() == 'petrol').sum()),
    hybrid=('fuel_type', lambda x: (x.str.lower() == 'hybrid').sum()),
    electric=('fuel_type', lambda x: (x.str.lower() == 'electric').sum()),
    europe=('region', lambda x: (x.str.lower() == 'europe').sum()),
    asia=('region', lambda x: (x.str.lower() == 'asia').sum()),
    america=('region', lambda x: (x.str.lower() == 'north america').sum()),
    car_count=('price_max', 'count')
)

In [20]:
df.to_csv("cars_dataset_2025_cleaned.csv", index=False)